In [1]:
import cv2
import mediapipe as mp
import numpy as np
#mediapipe객체 생성
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose
#비디오 캡쳐
video_path="test.mp4"
cap = cv2.VideoCapture("test.mp4")
cap_video=cv2.VideoCapture(video_path)

# 선택된 랜드마크 리스트
SELECTED_LANDMARKS = [
    'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW',
    'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
    'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE',
    'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX'
]
#1~33까지 인덱스 중에서 위에서 고른 랜드마크의 고유 인덱스를 얻어서 S_L_I 변수에 저장
SELECTED_LANDMARK_INDICES = [mp_pose.PoseLandmark[landmark].value for landmark in SELECTED_LANDMARKS]

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)
#관절의 각도 계산 b관절을 기준으로 a관절과 c관절의 cos값을 계산. cos값이 -1~1까지 나오므로 코사인의 역함수 값을 취하면 0~180도가 나옴. cos(-1): 180 cos(0):90
#cos (0도) = 1   cos(90도) =0 cos (180도) = -1


def process_frame(frame, pose):
    frame.flags.writeable = False
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame)
    frame.flags.writeable = True
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        connections_to_draw = [
            connection for connection in mp_pose.POSE_CONNECTIONS
            if connection[0] in SELECTED_LANDMARK_INDICES and connection[1] in SELECTED_LANDMARK_INDICES
        ]
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, connections_to_draw,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )

    return frame, results
#관절의 좌표값을 얻어서 angles리스트에 담아 cal angles함수로 보냄
def get_angles(landmarks):
    angles = []
    
    # 오른쪽 팔꿈치 각도
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
    angles.append(angle)

    # 왼쪽 팔꿈치 각도
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    angles.append(angle)
    
    # 오른쪽 무릎 각도
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    angle = calculate_angle(right_hip, right_knee, right_ankle)
    angles.append(angle)  # 수정: 튜플 대신 숫자만 추가

    # 왼쪽 무릎 각도
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    angle = calculate_angle(left_hip, left_knee, left_ankle)
    angles.append(angle)  # 수정: 튜플 대신 숫자만 추가


    # 오른쪽 발목 각도
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
    angle = calculate_angle(right_knee, right_ankle, right_foot_index)
    angles.append(angle)

    # 왼쪽 발목 각도
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
    angle = calculate_angle(left_knee, left_ankle, left_foot_index)
    angles.append(angle)

    # 오른쪽 골반 각도
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    angle = calculate_angle(right_shoulder, right_hip, right_knee)
    angles.append(angle)

    # 왼쪽 골반 각도
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    angle = calculate_angle(left_shoulder, left_hip, left_knee)
    angles.append(angle)

    return angles

# 메인 루프 부분
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened() and cap_video.isOpened():
        ret, frame = cap.read()
        ret_video, frame_video = cap_video.read()

        if not ret or not ret_video:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        frame, results = process_frame(frame, pose)
        frame_video, results_video = process_frame(frame_video, pose)

        if results.pose_landmarks and results_video.pose_landmarks:
            angles_webcam = get_angles(results.pose_landmarks.landmark)
            angles_video = get_angles(results_video.pose_landmarks.landmark)
            
            angle_differences = np.array(angles_webcam) - np.array(angles_video)
            
            # 각도 차이를 화면에 표시
            joint_names = ["Right Elbow", "Left Elbow", "Right Knee", "Left Knee", 
               "Right Ankle", "Left Ankle", "Right Hip", "Left Hip"]
            for i, (name, diff) in enumerate(zip(joint_names, angle_differences)):
                cv2.putText(frame, f"{name}: {diff:.2f}", 
                (10, 30 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            # 각도 차이 배열 출력
            print(f"Angle differences: {angle_differences}")

        # 이미지 크기 맞추기
        height = min(frame.shape[0], frame_video.shape[0])
        width = min(frame.shape[1], frame_video.shape[1])

        frame_resized = cv2.resize(frame, (width, height))
        frame_video_resized = cv2.resize(frame_video, (width, height))

        # 두 영상을 나란히 표시
        combined_frame = np.hstack((frame_resized, frame_video_resized))
        cv2.imshow('Webcam and Video Comparison', combined_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cap_video.release()
cv2.destroyAllWindows()
#같은 영상으로 관절 각도의 차이를 봤을때 이론상 0이 나와야 맞지만 개개인의 키와 특성을 고려했을때 아래 수치는 오차값이 작은 수준이므로 위의 각도 계산식으로 운동법 피드백을 위한 참고 자료로 사용가능하다고 판단.
#openpose와 비교했을 때 mediapipe의 실시간 추적이 더 빠르기때문에 실시간 웹캠과 비교하기에 더 적합하다고 판단하여 mediapipe사용
#openpose는 bottom-up방식으로 인물이 늘어나도 비슷한 계산시간을 소요하지만
#mediapipe는 top-down방식으로 인물 객체를 먼저 찾고 관절로 내려가기 떄문에 인물수에 따라 계산 시간이 선형적으로 증가

C:\Users\master\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Angle differences: [-14.17883637  66.05499934   3.80521103   3.09183451  -1.8385351
  -1.43294145   1.23905482  -1.74768803]
Angle differences: [-47.15775505  -6.22205688   0.77813     -0.93587864   9.67765493
  14.90766427   0.38520768   0.20727048]
Angle differences: [-64.7352166   44.14946202   0.776434     1.0712688    2.2723126
  10.64602278   0.5042147    0.464559  ]
Angle differences: [ 8.19518989e+01  5.39816136e+01  2.21459926e-01 -2.59834370e-01
 -1.32997571e+00 -1.86447994e+00 -3.30747515e-01 -5.26560163e-02]
Angle differences: [ 6.78798468 16.3839078   1.92607535 -0.21105477  0.52774108  9.71137202
  0.73601823 -0.47042289]
Angle differences: [ 8.54047906e+01 -2.06036606e+01 -1.21898467e-01  1.19007698e-01
  3.43099581e+00  3.40829794e+00 -4.07176573e-01 -8.03631888e-02]
Angle differences: [16.12973525  1.12690078  0.26811451  0.20569932  2.33016969  2.45306413
  0.05354502 -0.05209131]
Angle differences: [-36.90356863  26.40135543  -0.71156025  -0.78860526   1.82913118
  -